In [26]:
import polars as pl
import scipy.sparse as sp
import numpy as np
from tqdm import tqdm

In [3]:
%%time
df = pl.read_parquet("../data/combinations/k=2/mapped/0.pq")

CPU times: user 1.47 s, sys: 599 ms, total: 2.07 s
Wall time: 2.06 s


In [21]:
n_users = 0
max_comb_id = 0
for i in tqdm(range(42)):
    batch = pl.read_parquet(f"../data/combinations/k=2/mapped/{i}.pq")
    n_users += batch["user_id"].n_unique()
    max_comb_id = max(
        max_comb_id,
        batch
        .select(pl.col("url_combination_id").str.split(" "))
        .to_series()
        .explode()
        .cast(pl.Int32)
        .max()
    )

100%|███████████████████████████████████████████| 42/42 [12:02<00:00, 17.21s/it]


In [22]:
max_comb_id

163791897

In [23]:
n_users

415317

In [ ]:
combinations_matrix = sp.dok_matrix((n_users, max_comb_id), dtype=np.int32)
for i in tqdm(range(42)):
    batch = pl.read_parquet(f"../data/combinations/k=2/mapped/{i}.pq")
    for user_id, combs in batch.iter_rows():
        for comb in map(int, combs.split()):
            combinations_matrix[user_id, comb] += 1

  5%|█▊                                     | 2/42 [42:29<14:15:00, 1282.50s/it]

In [29]:
for user_id, combs in batch.iter_rows():
    break

In [30]:
combs

'8079160 8133131 4021875 10904004 5898178 856450 10993435 3943900 5331456 12875331 82677 322875 5593876 2365432 12071475 3572139 11026401 12852864 3781541 3121923 8084821 12676196 203582 7649299 9501907 6806162 4405606 10265786 9775834 4862777 8579290 3889453 11583574 4995067 3715927 4208081 968526 5135811 11284955 3925583 3110602 12180367 9725547 4746696 12940433 1008225 9588976 2941376 4387574 410343 1560473 2154898 5254950 11248678 842233 5445665 6139940 1011237 12359314 12004931 11569824 3776484 2631272 9696982 5631782 11621285 10091916 12892673 8513358 2624726 1624339 4549812 5539046 9277190 3130887 11928573 6365137 2775813'